# **Extracción, Transformación y Carga** (`ETL`) 

#### Teniendo en cuenta que en la Argentina, los accidentes de tránsito que arrojan víctimas fatales se caratulan como 'HOMICIDIO CULPOSO' y que los que no arrojan víctimas fatales se clasifican según su gravedad en 'LESIONES' o 'LESIONES GRAVES' (cuando el tiempo de recuperación demande mas de 30 días de reposo, o cuando la víctima haya sufrido la amputación de un miembro o la pérdida de un sentido), procederemos a realizar las normalizaciones de datos por separado para luego unificarlos en un único dataset.

## Importamos librerías y cargamos los archivos de entrada

In [69]:
import numpy as np
import pandas as pd

In [70]:

#Leemos los Dataset desde la Carpeta 'Datasets' donde se encuentra alojados los archivos
homicidios = pd.read_excel('Datasets/homicidios.xlsx',sheet_name = 'HECHOS',na_values=['SD', 'sd'])
victima_h = pd.read_excel('Datasets/homicidios.xlsx',sheet_name = 'VICTIMAS',na_values=['SD', 'sd'])
lesiones = pd.read_excel('Datasets/lesiones.xlsx' , sheet_name = 'HECHOS',na_values=['SD', 'sd'])
victima_l = pd.read_excel('Datasets/lesiones.xlsx' , sheet_name = 'VICTIMAS',na_values=['SD', 'sd'])

In [71]:
homicidios.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4.0,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1.0,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7.0,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0.0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,NaN
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5.0,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


#### Comenzamos con la exploración de los Datasets para tener un analisis previo y acercamiento mas amplio con los datos que vamods a trabajar, para ello usamos la funcion info nos muestra cantidas de columnas, nombres de columnas y tipos de datos

In [72]:
homicidios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID                     696 non-null    object        
 1   N_VICTIMAS             696 non-null    int64         
 2   FECHA                  696 non-null    datetime64[ns]
 3   AAAA                   696 non-null    int64         
 4   MM                     696 non-null    int64         
 5   DD                     696 non-null    int64         
 6   HORA                   695 non-null    object        
 7   HH                     695 non-null    float64       
 8   LUGAR_DEL_HECHO        695 non-null    object        
 9   TIPO_DE_CALLE          696 non-null    object        
 10  Calle                  695 non-null    object        
 11  Altura                 129 non-null    float64       
 12  Cruce                  525 non-null    object        
 13  Direc

In [73]:
victima_h.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  706 non-null    object        
 6   VICTIMA              708 non-null    object        
 7   SEXO                 711 non-null    object        
 8   EDAD                 664 non-null    float64       
 9   FECHA_FALLECIMIENTO  649 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 56.1+ KB


In [74]:
lesiones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     23785 non-null  object 
 1   n_victimas             23785 non-null  int64  
 2   aaaa                   23785 non-null  int64  
 3   mm                     23785 non-null  int64  
 4   dd                     23785 non-null  int64  
 5   fecha                  23785 non-null  object 
 6   hora                   23776 non-null  object 
 7   franja_hora            23776 non-null  float64
 8   direccion_normalizada  12917 non-null  object 
 9   comuna                 22770 non-null  object 
 10  tipo_calle             12740 non-null  object 
 11  otra_direccion         5490 non-null   object 
 12  calle                  12867 non-null  object 
 13  altura                 12771 non-null  float64
 14  cruce                  9407 non-null   object 
 15  ge

In [75]:
victima_l.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID hecho          27605 non-null  object        
 1   AAA               27605 non-null  int64         
 2   MM                27605 non-null  int64         
 3   DD                27605 non-null  int64         
 4   FECHA             27605 non-null  datetime64[ns]
 5   VEHICULO_VICTIMA  11333 non-null  object        
 6   SEXO              25377 non-null  object        
 7   EDAD_VICTIMA      23134 non-null  float64       
 8   GRAVEDAD          537 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 1.9+ MB


### DATASET _`Homicidios`_

#### Se Eliminan las columnas que no consumen relevancia para el analisis.

In [76]:
homicidios.drop(columns=['AAAA', 'MM', 'DD', 'HORA', 'LUGAR_DEL_HECHO', 'Calle', 
                        'Altura', 'Cruce', 'Dirección Normalizada', 'PARTICIPANTES'], inplace=True)

In [77]:
#Volvemos a usar la función info aplicando al dataset, para corroborar la eliminancíon de las columnas
homicidios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ID             696 non-null    object        
 1   N_VICTIMAS     696 non-null    int64         
 2   FECHA          696 non-null    datetime64[ns]
 3   HH             695 non-null    float64       
 4   TIPO_DE_CALLE  696 non-null    object        
 5   COMUNA         696 non-null    int64         
 6   XY (CABA)      696 non-null    object        
 7   pos x          696 non-null    object        
 8   pos y          696 non-null    object        
 9   VICTIMA        687 non-null    object        
 10  ACUSADO        673 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(7)
memory usage: 59.9+ KB


In [78]:
# Verificamos cantidad de nulos
homicidios.isnull().sum()

ID                0
N_VICTIMAS        0
FECHA             0
HH                1
TIPO_DE_CALLE     0
COMUNA            0
XY (CABA)         0
pos x             0
pos y             0
VICTIMA           9
ACUSADO          23
dtype: int64

In [79]:
# Se Reemplaza la columna 0 a nulos
homicidios.loc[homicidios['COMUNA'] == '0' ,'COMUNA'] = np.nan

# Se Reemplaza los valores "." por nulos en latitud y longitud
homicidios.loc[homicidios['pos x'] == '.' ,'pos x'] = np.nan
homicidios.loc[homicidios['pos y'] == '.' ,'pos y'] = np.nan
homicidios.loc[homicidios['XY (CABA)'] == 'Point (. .)' ,'XY (CABA)'] = np.nan

In [80]:
# Ahora Cambiamos el tipo de dato de las columnas
homicidios['HH'] = homicidios['HH'].astype('category')
homicidios['TIPO_DE_CALLE'] = homicidios['TIPO_DE_CALLE'].astype('category')
homicidios['COMUNA'] = homicidios['COMUNA'].astype('category')
homicidios['VICTIMA'] = homicidios['VICTIMA'].astype('category')
homicidios['ACUSADO'] = homicidios['ACUSADO'].astype('category')

# Y Tambien Cambiamos el tipo de dato de las columnas
homicidios['pos x'] = homicidios['pos x'].astype('float')
homicidios['pos y'] = homicidios['pos y'].astype('float')

In [81]:
# Renombramos las columnas
nuevos_nombres = {'ID': 'ID_HECHO','pos x':'LONGITUD','pos y':'LATITUD',
                  'HH':'HORA','VICTIMA':'VEHICULO_VICTIMA','ACUSADO':'VEHICULO_ACUSADO',
                  'Rango_etario ':'RANGO_ETARIO'}
                 
# Reemplazamos nombres de columnas
homicidios = homicidios.rename(columns=nuevos_nombres)

# Agregamos la columna gravedad para diferenciar los homicidios de las lesiones
homicidios["GRAVEDAD"]="FATAL"

# Cambiamos el tipo de dato de la columna FECHA a datetime
homicidios["FECHA"]=pd.to_datetime(homicidios["FECHA"], format='%y/%m/%d', errors='coerce').dt.date

#### Columna 'TIPO_DE_CALLE'

In [82]:
# Verificamos el contenido de la columna TIPO_DE_CALLE
homicidios['TIPO_DE_CALLE'].value_counts()

AVENIDA      429
CALLE        136
AUTOPISTA     66
GRAL PAZ      65
Name: TIPO_DE_CALLE, dtype: int64

In [83]:
homicidios['TIPO_DE_CALLE'].replace('GRAL PAZ', 'AVENIDA', inplace=True) # Reemplazamos GRAL PAZ por AVENIDA

In [84]:
# Volvemos a verificar el contenido de la columna TIPO_DE_CALLE
homicidios['TIPO_DE_CALLE'].value_counts()

AVENIDA      494
CALLE        136
AUTOPISTA     66
Name: TIPO_DE_CALLE, dtype: int64

In [85]:
homicidios.columns

Index(['ID_HECHO', 'N_VICTIMAS', 'FECHA', 'HORA', 'TIPO_DE_CALLE', 'COMUNA',
       'XY (CABA)', 'LONGITUD', 'LATITUD', 'VEHICULO_VICTIMA',
       'VEHICULO_ACUSADO', 'GRAVEDAD'],
      dtype='object')

In [86]:
# Creamos la columna CARATULA POLICIAL y la rellenamos con HOMICIDIO CULPOSO
homicidios['CARATULA_POLICIAL'] = 'HOMICIDIO CULPOSO'

In [87]:
homicidios['CARATULA_POLICIAL'].value_counts()

HOMICIDIO CULPOSO    696
Name: CARATULA_POLICIAL, dtype: int64

### DATASET _`Homicidios (víctimas)`_

In [88]:
#Aplicamos la función info para visualizar #de columnnas, nombres de columnas y tipo de datos.
victima_h.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  706 non-null    object        
 6   VICTIMA              708 non-null    object        
 7   SEXO                 711 non-null    object        
 8   EDAD                 664 non-null    float64       
 9   FECHA_FALLECIMIENTO  649 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 56.1+ KB


#### Procedemos a eliminar las columnas que no aportan información para el analisis

In [89]:
victima_h.drop(columns=['AAAA', 'MM', 'DD', 'FECHA_FALLECIMIENTO'], inplace=True)

# Agregamos la columna gravedad para diferenciar los homicidios de las lesiones
victima_h['GRAVEDAD'] = 'FATAL'

In [90]:
# Creamos una función para asignar el rango etario

def asignar_rango_etario(edad):
    rangos_etarios = {
        (0, 17): "1_ 0 a 17 años",
        (18, 29): "2_ 18 a 29 años",
        (30, 39): "3_ 30 a 39 años",
        (40, 49): "4_ 40 a 49 años",
        (50, 59): "5_ 50 a 59 años",
        (60, float('inf')): "6_ 60 años o más"
    }

    for rango, etiqueta in rangos_etarios.items():
        if rango[0] <= edad <= rango[1]:
            return etiqueta

    return np.nan

In [91]:
# Cambiamos los SD por nan
victima_h.loc[victima_h['EDAD'] == 'SD', 'EDAD'] = np.nan
victima_h.loc[victima_h['EDAD'] == 'sd', 'EDAD'] = np.nan

In [92]:
# Cambiamos el tipo de dato de la columna EDAD
victima_h['EDAD'] = victima_h['EDAD'].astype('float')

In [93]:
# Aplicamos la funcion para asignar el rango etario
victima_h['RANGO_ETARIO'] = victima_h['EDAD'].apply(lambda x: asignar_rango_etario(x))
victima_h.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ID_hecho      717 non-null    object        
 1   FECHA         717 non-null    datetime64[ns]
 2   ROL           706 non-null    object        
 3   VICTIMA       708 non-null    object        
 4   SEXO          711 non-null    object        
 5   EDAD          664 non-null    float64       
 6   GRAVEDAD      717 non-null    object        
 7   RANGO_ETARIO  664 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 44.9+ KB


In [94]:
# Cambiamos el nombre de las columnas  
nuevos_nombres = {'ID_hecho':'ID_HECHO','ROL':'ROL_VICTIMA','VICTIMA':'VEHICULO_VICTIMA','SEXO':'SEXO_VICTIMA','EDAD':'EDAD_VICTIMA'}

# Procedemos a reemplazar
victima_h = victima_h.rename(columns=nuevos_nombres)
victima_h.columns

Index(['ID_HECHO', 'FECHA', 'ROL_VICTIMA', 'VEHICULO_VICTIMA', 'SEXO_VICTIMA',
       'EDAD_VICTIMA', 'GRAVEDAD', 'RANGO_ETARIO'],
      dtype='object')

In [95]:
#Aplicamos counts, a la columna Rol Victima para visualizar cuantos datos tiene cada valor de esta columna.
victima_h['ROL_VICTIMA'].value_counts()

CONDUCTOR               330
PEATON                  267
PASAJERO_ACOMPAÑANTE     80
CICLISTA                 29
Name: ROL_VICTIMA, dtype: int64

In [96]:
# Intercambiamos el valor 'PASAJERO_ACOMPAÑANTE' por 'PASAJERO' para unificar
victima_h['ROL_VICTIMA'] = victima_h['ROL_VICTIMA'].replace('PASAJERO_ACOMPAÑANTE','PASAJERO')

# Cambiamos el tipo de dato de la columna FECHA a datetime
victima_h["FECHA"]=pd.to_datetime(victima_h["FECHA"], format='%y/%m/%d', errors='coerce').dt.date

### DATASET _`Lesiones`_

In [97]:
#Aplicamos la función info para visualizar #de columnnas, nombres de columnas y tipo de datos.
lesiones.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     23785 non-null  object 
 1   n_victimas             23785 non-null  int64  
 2   aaaa                   23785 non-null  int64  
 3   mm                     23785 non-null  int64  
 4   dd                     23785 non-null  int64  
 5   fecha                  23785 non-null  object 
 6   hora                   23776 non-null  object 
 7   franja_hora            23776 non-null  float64
 8   direccion_normalizada  12917 non-null  object 
 9   comuna                 22770 non-null  object 
 10  tipo_calle             12740 non-null  object 
 11  otra_direccion         5490 non-null   object 
 12  calle                  12867 non-null  object 
 13  altura                 12771 non-null  float64
 14  cruce                  9407 non-null   object 
 15  ge

In [98]:
#Procedemos a Eliminar las columnas que no suman al analisis
lesiones.drop(columns=['aaaa', 'mm', 'dd','hora','direccion_normalizada',
                          'calle','altura','cruce','participantes','moto',
                          'auto','transporte_publico','camion','otra_direccion',
                          'ciclista'], inplace=True)

In [99]:
# Convertimos a mayusculas el nombre de las columnas 
lesiones.columns = lesiones.columns.str.upper()
#y reemplazamos algunas con nueva etiqueta 
lesiones = lesiones.rename(columns={'ID':'ID_HECHO'})
lesiones = lesiones.rename(columns={'FRANJA_HORA':'HORA'})
lesiones = lesiones.rename(columns={'TIPO_CALLE':'TIPO_DE_CALLE'})
lesiones = lesiones.rename(columns={'TIPO_CALLE':'TIPO_DE_CALLE'})
lesiones = lesiones.rename(columns={'GEOCODIFICACION_CABA':'XY (CABA)'})
lesiones = lesiones.rename(columns={'VICTIMA':'VEHICULO_VICTIMA'})
lesiones = lesiones.rename(columns={'ACUSADO':'VEHICULO_ACUSADO'})
lesiones = lesiones.rename(columns={'LATUTID':'LATITUD'})

### Columna 'ID_HECHO'

In [100]:
# Modificamos la composición de la columna ID_HECHO para asemejarla a la de homicidios
lesiones['ID_HECHO'] = lesiones['ID_HECHO'].str.replace('LC-', '').str.strip()

### Columna 'FECHA'

In [101]:
# Hacemos cambios a la columna fecha
lesiones['FECHA'].replace("'","", inplace=True)
lesiones['FECHA'] = lesiones['FECHA'].astype('datetime64[ns]')

# Cambiamos el formato de la fecha
lesiones["FECHA"]=pd.to_datetime(lesiones["FECHA"], format='%y/%m/%d', errors='coerce').dt.date

#### Columna 'COMUNA'

In [102]:
lesiones['COMUNA'].value_counts()

1                  2526
15                 1908
4                  1729
3                  1718
9                  1674
14                 1581
7                  1516
12                 1440
11                 1411
13                 1355
10                 1347
5                  1226
8                  1062
6                   957
2                   878
No Especificada     442
Name: COMUNA, dtype: int64

In [103]:
# Remplazamos "No especificada" por nulos
lesiones.loc[lesiones['COMUNA'] == 'No Especificada', 'COMUNA'] = np.nan

#### Columna 'GRAVEDAD'

In [104]:
lesiones['GRAVEDAD'].value_counts() # Revisamos los valores de la columna gravedad

GRAVE    642
FATAL      3
grave      1
Name: GRAVEDAD, dtype: int64

In [105]:
lesiones['GRAVEDAD'] = lesiones['GRAVEDAD'].str.upper() # Pasamos a mayusculas la columna gravedad

#### Conforme al diccionario proporcionado, los valores equivalentes a SD son en su mayoria de gravedad leve y teniendo en cuenta que en un principio convertimos los datos SD, pasamos los nulos a LEVE.

In [106]:
lesiones.loc[lesiones['GRAVEDAD'].isnull(), 'GRAVEDAD'] = 'LEVE'

In [107]:
lesiones.columns

Index(['ID_HECHO', 'N_VICTIMAS', 'FECHA', 'HORA', 'COMUNA', 'TIPO_DE_CALLE',
       'XY (CABA)', 'LONGITUD', 'LATITUD', 'VEHICULO_VICTIMA',
       'VEHICULO_ACUSADO', 'GRAVEDAD'],
      dtype='object')

#### Columna 'TIPO_DE_CALLE'

In [108]:
lesiones['TIPO_DE_CALLE'].value_counts()

AVENIDA      6775
CALLE        5383
GRAL PAZ      538
AUTOPISTA      44
Name: TIPO_DE_CALLE, dtype: int64

In [109]:
lesiones['TIPO_DE_CALLE'].replace('GRAL PAZ', 'AVENIDA', inplace=True) # Reemplazamos GRAL PAZ por AVENIDA

In [110]:
lesiones['TIPO_DE_CALLE'].value_counts() # Verificamos el contenido de la columna TIPO_DE_CALLE

AVENIDA      7313
CALLE        5383
AUTOPISTA      44
Name: TIPO_DE_CALLE, dtype: int64

#### Columna 'GRAVEDAD'

In [111]:
lesiones['GRAVEDAD'].value_counts() # Verificamos el contenido de la columna GRAVEDAD

LEVE     23139
GRAVE      643
FATAL        3
Name: GRAVEDAD, dtype: int64

In [112]:
# Creamos una función que nos permita generar la columna CARATULA POLICIAL
def generar_caratula(gravedad):
    if gravedad == 'LEVE':
        return 'LESIONES'
    elif gravedad == 'GRAVE':
        return 'LESIONES GRAVES'
    elif gravedad == 'FATAL':
        return 'HOMICIDIO CULPOSO'

In [113]:
lesiones['CARATULA_POLICIAL'] = lesiones['GRAVEDAD'].apply(generar_caratula)

In [114]:
lesiones['CARATULA_POLICIAL'].value_counts()

LESIONES             23139
LESIONES GRAVES        643
HOMICIDIO CULPOSO        3
Name: CARATULA_POLICIAL, dtype: int64

### _`Lesiones (víctimas)`_

In [115]:
# Pasamos las columnas a mayusculas
victima_l.columns = victima_l.columns.str.upper()

# Cambiamos nombres a las columnas para coincidir con el df_homicidios
victima_l = victima_l.rename(columns={'ID HECHO':'ID_HECHO','FECHA ':'FECHA','SEXO':'SEXO_VICTIMA'})

# Cambiamos el formato de la fecha
victima_l["FECHA"]=pd.to_datetime(victima_l["FECHA"], format='%y/%m/%d', errors='coerce').dt.date

victima_l.columns

Index(['ID_HECHO', 'AAA', 'MM', 'DD', 'FECHA', 'VEHICULO_VICTIMA',
       'SEXO_VICTIMA', 'EDAD_VICTIMA', 'GRAVEDAD'],
      dtype='object')

In [116]:
# Eliminamos las columnas que no se van a usar
victima_l.drop(columns=['AAA', 'MM', 'DD'], inplace=True)

# Columnas
victima_l.columns

Index(['ID_HECHO', 'FECHA', 'VEHICULO_VICTIMA', 'SEXO_VICTIMA', 'EDAD_VICTIMA',
       'GRAVEDAD'],
      dtype='object')

In [117]:
# Creamos la columna ROL_VICTIMA con nulls
victima_l['ROL_VICTIMA'] = np.nan

# remplazar los nulos de la columna GRAVEDAD por "LEVEMENTE HERIDO"
victima_l.loc[victima_l['GRAVEDAD'].isnull(), 'GRAVEDAD'] = 'LEVEMENTE HERIDO'

In [118]:
# Rango Etario
victima_l['EDAD_VICTIMA'] = victima_l['EDAD_VICTIMA'].astype('float')
victima_l['RANGO_ETARIO'] = victima_l['EDAD_VICTIMA'].apply(lambda x: asignar_rango_etario(x))

In [119]:
victima_l['SEXO_VICTIMA'].value_counts()

Varon     16796
Mujer      8246
varon       226
MUJER        62
mujer        46
Mujer         1
Name: SEXO_VICTIMA, dtype: int64

In [120]:
victima_l['SEXO_VICTIMA'] = victima_l['SEXO_VICTIMA'].str.upper() # Pasamos a mayusculas la columna 'SEXO_VICTIMA

In [121]:
victima_l['SEXO_VICTIMA'] = victima_l['SEXO_VICTIMA'].str.strip() # Eliminamos los espacios en blanco

In [122]:
victima_l.loc[victima_l['SEXO_VICTIMA'] == 'MUJER', 'SEXO_VICTIMA'] = 'FEMENINO'
victima_l.loc[victima_l['SEXO_VICTIMA'] == 'VARON', 'SEXO_VICTIMA'] = 'MASCULINO'

In [123]:
victima_l['SEXO_VICTIMA'].unique()

array(['MASCULINO', nan, 'FEMENINO'], dtype=object)

#### Columna 'ID_HECHO'

In [124]:
# Modificamos la composición de la columna ID_HECHO para asemejarla a la de víctimas de accidentes fatales
victima_l['ID_HECHO'] = victima_l['ID_HECHO'].str.replace('LC-', '').str.strip()

#### Columna 'GRAVEDAD'

In [125]:
victima_l['GRAVEDAD'].value_counts()

LEVEMENTE HERIDO    27068
GRAVE                 536
grave                   1
Name: GRAVEDAD, dtype: int64

In [126]:
victima_l['GRAVEDAD'] = victima_l['GRAVEDAD'].str.upper() # Pasamos a mayusculas la columna gravedad

In [127]:
victima_l['GRAVEDAD'] = victima_l['GRAVEDAD'].replace('LEVEMENTE HERIDO', 'LEVE') # Reemplazamos LEVEMENTE HERIDO por LEVE

#### Revisamos las 2 primeras filas de nuestros datasets

In [128]:
homicidios.head(2)


,ID_HECHO,N_VICTIMAS,FECHA,HORA,TIPO_DE_CALLE,COMUNA,XY (CABA),LONGITUD,LATITUD,VEHICULO_VICTIMA,VEHICULO_ACUSADO,GRAVEDAD,CARATULA_POLICIAL
0,2016-0001,1,2016-01-01,4.0,AVENIDA,8.0,Point (98896.78238426 93532.43437792),-58.475340,-34.687570,MOTO,AUTO,FATAL,HOMICIDIO CULPOSO
1,2016-0002,1,2016-01-02,1.0,AVENIDA,9.0,Point (95832.05571093 95505.41641999),-58.508775,-34.669777,AUTO,PASAJEROS,FATAL,HOMICIDIO CULPOSO


In [129]:
victima_h.head(2)

,ID_HECHO,FECHA,ROL_VICTIMA,VEHICULO_VICTIMA,SEXO_VICTIMA,EDAD_VICTIMA,GRAVEDAD,RANGO_ETARIO
0,2016-0001,2016-01-01,CONDUCTOR,MOTO,MASCULINO,19.0,FATAL,2_ 18 a 29 años
1,2016-0002,2016-01-02,CONDUCTOR,AUTO,MASCULINO,70.0,FATAL,6_ 60 años o más


In [130]:
lesiones.head(2)

,ID_HECHO,N_VICTIMAS,FECHA,HORA,COMUNA,TIPO_DE_CALLE,XY (CABA),LONGITUD,LATITUD,VEHICULO_VICTIMA,VEHICULO_ACUSADO,GRAVEDAD,CARATULA_POLICIAL
0,2019-0000179,1,2019-01-01,9.0,14,NaN,Point (104991.46333918849995825 107720.9664748...,-58.408911,-34.559658,CICLISTA,NaN,LEVE,LESIONES
1,2019-0000053,1,2019-01-01,1.0,8,NaN,Point (101813.84712503915943671 95578.55507230...,-58.443510,-34.669125,AUTO,NaN,LEVE,LESIONES


In [131]:
victima_l.head(2)

,ID_HECHO,FECHA,VEHICULO_VICTIMA,SEXO_VICTIMA,EDAD_VICTIMA,GRAVEDAD,ROL_VICTIMA,RANGO_ETARIO
0,2019-0000053,2019-01-01,NaN,MASCULINO,57.0,LEVE,NaN,5_ 50 a 59 años
1,2019-0000063,2019-01-01,NaN,NaN,NaN,LEVE,NaN,NaN


#### Realizamos unas últimas modificaciones para unificar los datasets de Homicidios y Lesiones en un único dataset llamado 'accidentes', mientras que los datasets de víctimas de homicidios culposos y los de lesiones los unificaremos en un único dataset llamado 'victimas'

### _`Accidentes`_

In [132]:
homicidios['FECHA'] = pd.to_datetime(homicidios['FECHA'], errors='coerce')
lesiones['FECHA'] = pd.to_datetime(lesiones['FECHA'], errors='coerce')

In [133]:
accidentes = pd.concat([homicidios, lesiones], ignore_index=True)

In [134]:
accidentes.to_excel('Datasets/accidentes.xlsx', index=False)

In [135]:
accidentes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24481 entries, 0 to 24480
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ID_HECHO           24481 non-null  object        
 1   N_VICTIMAS         24481 non-null  int64         
 2   FECHA              24481 non-null  datetime64[ns]
 3   HORA               24471 non-null  float64       
 4   TIPO_DE_CALLE      13436 non-null  object        
 5   COMUNA             23024 non-null  object        
 6   XY (CABA)          23215 non-null  object        
 7   LONGITUD           22998 non-null  float64       
 8   LATITUD            22998 non-null  float64       
 9   VEHICULO_VICTIMA   13739 non-null  object        
 10  VEHICULO_ACUSADO   9170 non-null   object        
 11  GRAVEDAD           24481 non-null  object        
 12  CARATULA_POLICIAL  24481 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(1), object(8)
memory 

### _`Víctimas`_

In [136]:
victima_h['FECHA'] = pd.to_datetime(victima_h['FECHA'], errors='coerce')
victima_l['FECHA'] = pd.to_datetime(victima_l['FECHA'], errors='coerce')

In [137]:
victimas = pd.concat([victima_h, victima_l], ignore_index=True)

In [138]:
victimas.to_excel('Datasets/victimas.xlsx', index=False)

In [139]:
victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28322 entries, 0 to 28321
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID_HECHO          28322 non-null  object        
 1   FECHA             28322 non-null  datetime64[ns]
 2   ROL_VICTIMA       706 non-null    object        
 3   VEHICULO_VICTIMA  12041 non-null  object        
 4   SEXO_VICTIMA      26088 non-null  object        
 5   EDAD_VICTIMA      23798 non-null  float64       
 6   GRAVEDAD          28322 non-null  object        
 7   RANGO_ETARIO      23798 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 1.7+ MB
